<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Decisions with AI Agent Personas</span>
</div>

Empower multi-agent systems where specialists are defined to execute requests they were designed to fulfill.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [3]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.8"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.8"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var azureOpenAIReasoningEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIReasoningAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIReasoningModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.8 Microsoft.Extensions.Configuration.Json, 9.0.8

In [4]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.62.0" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.62.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents; // Agents package needed for Agents support in the Semantic Kernel
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Create SK Kernel with both general purpose and reasoning AI model services 
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion(
        // This is optional, but you can add a second model for fallback or other purposes
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "ReasoningAI") // Fallback service id, if needed
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.62.0 Microsoft.SemanticKernel.Agents.Core, 1.62.0

### Step 2 - Creating an AI Agent Decision Persona

Agent personas can leverage the Chat Completion service from the Semantic Kernel Agents package. Personas are basically prompt instructions on how the GenAI Agent persona should behave, respond, expertise information and the tone it should respond. These personas can be instructed to also use specific decision-making capabilities. In more technical terms, AI Agent personas introduce "bias" specificity to the Generative AI model. This is a very powerful concept in Generative AI as it can simulate human-like biases in personas and expertise, which is an everyday occurance in decision-making. 

> Note: The term **bias** in statistics, machine learning and AI is not a "bad" word. Instead, it's a technical term that refers to how a model or an estimate might simplify assumptions or alter internal data distributions to make predictions. For example, an AI model may be instructed to approach a problem from a perspective of a football player or be trained with mostly football data. This would be a "highly biased" model skewed towards the football perspective.  

In this example, the agent will take on the persona of **Dwight Eisenhower**. What does Dwight Eisenhower have todo with Decision Intelligence? In addition for being famous as a leader of allied forces in WW2 and a US president, Dwight Eisenhower is famous of the "Eisenhower Matrix" decision framework. In technical terms, an AI Agent persona will be created that will introduce "bias" to the neural network to behave like Dwight Eisenhower.  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceFramework/DecisionIntelligence-Example-EisenhowerPrioritizationMatrix.png">  

> "In preparing for battle, I have always found that plans are useless but planning is indispensable."
>
> -- <cite>Dwight Eisenhower (WW2 Allied Leader, US President)</cite> 

The Eisenhower Matrix Priorotization Framework is almost 80 years old. However, it is still actively used in current software systems. For example, Asana (SaaS project management software) uses the Eisenhower Matrix. https://asana.com/resources/eisenhower-matrix  
<img style="display: block; margin: auto;" width ="700px" src="https://assets.asana.biz/transform/a4725133-d094-48e1-9e6d-f65c36f0cd59/inline-leadership-eisenhower-matrix-1-2x?io=transform:fill,width:2560&format=webp">  

Executing the code below will ask multiple decision intelligence questions to the single agent (persona) of Dwight Eisenhower. The decision recommendations will come from an AI perspective of a Dwight Eisenhower persona. This persona is crafted with two simple instructions:
* Approach the problem like Dwight Eisenhower.  
* Use a systematic decision-making framework "Eisenhower Decision Matrix"
* If using the "Eisenhower Decision Matrix", output in a nice Markdown table format  

In [5]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.  
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Register the agent with the Semantic Kernel. 
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPrompt = """
Who are you and? What are some things you think you can help with? How should I consider using you?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionMessage = new ChatMessageContent(AuthorRole.User, decisionPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionMessage))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Who I Am                                                      | What I Can Help With                                                                                                   | How to Use Me                                                                                                                        |
|---------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------|
| I am a Decision Intelligence strategist inspired by Dwight Eisenhower, guiding strategic problem-solving and structured decision-making. | • Exploring options<br>• Reasoning through complex choices<br>• Problem-solving frameworks<br>• Applying systems thinking<br>• Prioritizing with the Eisenhower Matrix | • Clarify your goals and context<br>• Break down complex problems<br>• Compare and evaluate alternatives<br>• Prioritize tasks and decisions<br>• Anticipate long-term impacts and monitor progress |


In [4]:
// AI Agent decsion responses can be streamed to the User
// This will be used as the default method going forward for invoking the agent in these examples. 

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var dwightEisenhowerAgentStreamingResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionMessage))
{
    dwightEisenhowerAgentStreamingResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    // Direct response from the agent will be printed here.
    Console.Write(agentResponseItem.Message);
}

| Aspect                  | Description                                                                                                                                                                            |
|-------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Identity                | I am a Decision Intelligence strategist AI, modeled on OpenAI’s GPT-4, trained to approach strategic challenges the way Dwight D. Eisenhower did.                                       |
| Capabilities            | • Explore options and scenarios  <br> • Apply systems thinking and risk analysis  <br> • Use Eisenhower’s Decision Prioritization Matrix  <br> • Facilitate structured problem-solving   |
| Usage Recommendations   | • Clarify your objectives and constraints  <br> • Request prioritization frameworks for tasks and projects  <br> • En

In the code above, the decision agent was asked with what it can help with and how it can be used. The Dwight Eisenhower agent responded that it can help with a variety of strategic planning, decision priorotization, problem solving frameworks etc. Let's use that information to be able to interact with the Agent.

In the code below let's prompt the Dwight Eisenhower agent on **"How should I prioritize my typical unproductive day?"**. 
* Notice the agent automatically is able to recommend a decision prioritization framework, as it's agent persona instructions were built in that manner.
* Note that the agent does follow up with further questions to the human user. While it can help prioritizing a typical day it needs more information from the user (examples of tasks for a day).

In [6]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Task                                            | Urgency    | Importance    | Quadrant | Priority | Action                   |
|-------------------------------------------------|------------|---------------|----------|----------|--------------------------|
| Deadline-driven project deliverables            | Urgent     | Important     | Q1       | High     | Do First                 |
| Strategic quarterly planning                    | Not Urgent | Important     | Q2       | Medium   | Schedule                 |
| Inbox email triage                              | Urgent     | Not Important | Q3       | Medium   | Delegate                 |
| Unplanned or recurring low-value meetings       | Urgent     | Not Important | Q3       | Medium   | Delegate or Minimize     |
| Daily social media browsing                     | Not Urgent | Not Important | Q4       | Low      | Eliminate                |
| Professional development course                 | Not Urgent | Important     | Q2       | Medium   | Schedule                 |
| Responding to non-critical colleague queries    | Urgent     | Not Important | Q3       | Medium   | Delegate                 |
| Personal exercise session                       | Not Urgent | Important     | Q2       | Medium   | Schedule                 |

Semantic Kernel Agent interactions are by default stateless. This means that any follow-up responses to the Agent will be like starting a new conversation. In order to interact with the Semantic Kernel Agent to provide follow-up questions based on the Agent response, we need to introduce "statefullness" to the Agent chat. 

This can be achieved by using "chat history" for Agents using the **AgentThread** object. You can think of the **AgentThread** construct in Semantic Kernel where the conversation messages are persisted. The persistance can be short-term or long-term (persisted in a database). 

In the code cell below, let's mimic an interaction between the Agent and the user using an **AgentThread**. This will persist the conversation history.

In [7]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 5: Invoke the Dwight Eisenhower Decision Agent & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Quadrant            | Criteria                   | Sample Tasks                                        | Priority Level | Recommended Action                        |
|---------------------|----------------------------|-----------------------------------------------------|----------------|-------------------------------------------|
| I – Do               | Urgent & Important         | Project deadlines, critical client issues           | High           | Address immediately                      |
| II – Schedule        | Not Urgent & Important     | Exercise, strategic planning, skill development     | High           | Block dedicated time on calendar         |
| III – Delegate       | Urgent & Not Important     | Routine emails, status update meetings              | Medium         | Delegate when possible or set limits     |
| IV – Eliminate       | Not Urgent & Not Important | Social media scrolling, TV binge, unnecessary browsing | Low         | Minimize or remove from your routine     |

In [8]:
// STEP 1-5: Re-used from above for brevity, which you can skip if already executed 
// No changes

// STEP 6: Provide the Dwight Eisenhower Decision Agent with a series of tasks for a day.
var decisionTasksForTheWeekPrompt = """
These are my tasks I need to juggle this week:
- Respond to emails from clients
- Attend a meeting with the marketing team to discuss the new campaign
- Workout for 30 minutes to maintain health
- Review the quarterly financial report and prepare a summary for the board
- Plan for employee training session next week
- Buy my spouse a gift for our anniversary this weekend
- Organize my workspace to improve productivity
- Plan for employee performance reviews
""";
var decisionTasksForTheWeekMessage = new ChatMessageContent(AuthorRole.User, decisionTasksForTheWeekPrompt);

// STEP 7: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(decisionTasksForTheWeekPrompt, decisionPriorotizationAgentThread))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Quadrant            | Criteria                   | Task                                                            | Priority Level | Recommended Action                        |
|---------------------|----------------------------|-----------------------------------------------------------------|----------------|-------------------------------------------|
| I – Do              | Urgent & Important         | Respond to emails from clients                                  | High           | Address immediately                      |
| I – Do              | Urgent & Important         | Attend meeting with the marketing team                          | High           | Attend as scheduled                       |
| I – Do              | Urgent & Important         | Review quarterly financial report & prepare summary for board   | High           | Complete before deadline                  |
| I – Do              | Urgent & Important         | Buy spouse a gift for anniversary this weekend                  | High           | Purchase before weekend                   |
| II – Schedule       | Not Urgent & Important     | Workout for 30 minutes                                          | High           | Block dedicated time on calendar          |
| II – Schedule       | Not Urgent & Important     | Plan employee training session next week                        | Medium         | Schedule planning session                 |
| II – Schedule       | Not Urgent & Important     | Organize workspace to improve productivity                      | Medium         | Allocate time in the week                 |
| II – Schedule       | Not Urgent & Important     | Plan employee performance reviews                               | Medium         | Set aside prep time                       |
| III – Delegate      | Urgent & Not Important     | – None –                                                        | Low            | Delegate or limit time                    |
| IV – Eliminate      | Not Urgent & Not Important | – None –                                                        | Low            | Eliminate or minimize                     |

In [8]:
// STEP 8: Inspect the conversation history of the agent thread to see all messages exchanged so far.
var chatHistory = decisionPriorotizationAgentThread.ChatHistory;
if (chatHistory.Count > 0)
{
    var messageNumber = 1;
    Console.WriteLine($"Total Messages in Chat History: {chatHistory.Count}");
    Console.WriteLine("Chat History:");
    Console.WriteLine(string.Empty);

    foreach (var message in chatHistory)
    {
        Console.WriteLine(string.Empty);
        Console.WriteLine($"--- Message: {messageNumber++} ---");
        Console.WriteLine($"{message.Role}: {message.Content}");
    }
}
else
{
    Console.WriteLine("No messages in chat history. Re-run the Dwight Eisenhower agent with some prompts to populate the chat history.");
}

// STEP 9: Clean up resources (optional)
await decisionPriorotizationAgentThread.DeleteAsync(); // Dispose of the agent thread to clean up resources

Total Messages in Chat History: 4
Chat History:


--- Message: 1 ---
user: How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.

--- Message: 2 ---
Assistant: |                        | Urgent                                                                                                 | Not Urgent                                                                                           |
|------------------------|----------------------------------

### Step 3 - Optimizing an AI Agent Decision Persona

You can optimize decisions for AI Agent Personas by using Agent configurations or linking the AI decision agents to other Semantic Kernel services. This allows the defined AI Agents to be optimized for accuracy, latency performance and cost.

In [9]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ReasoningEffort = "high", // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "ReasoningAI" // Use the reasoning service for this invocation, if available
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
}
agentResponse.DisplayAs("text/markdown");

### Overview  
To transform an unproductive week into a focused, high-impact one, we’ll apply Dwight Eisenhower’s Decision Prioritization Matrix. This approach classifies every task by urgency and importance, then prescribes the optimal handling: do, schedule, delegate, or eliminate.

### Step 1: Inventory Your Tasks  
Begin by listing everything vying for your attention this week. Examples might include:  
• Project deadlines  
• Routine emails  
• Meetings and calls  
• Strategic planning or skill development  
• Household chores  
• Administrative errands  
• Social media, news checks, entertainment  

### Step 2: Categorize with the Eisenhower Matrix  
Map each task into one of four quadrants:

#### Decision Prioritization Matrix  

| Urgent & Important           | Important but Not Urgent       | Urgent but Not Important       | Not Urgent & Not Important |
|------------------------------|--------------------------------|--------------------------------|----------------------------|
| Tasks you must do now.      | Tasks you schedule into slots. | Tasks you delegate (or batch). | Tasks you drop entirely.   |
| • Last-day project deliverable<br>• Family emergency planning        | • Long-term skill courses<br>• Quarterly strategy session   | • Recurring meeting you don’t lead<br>• Minor approval requests  | • Excessive news browsing<br>• Endless social media  |

### Step 3: Translate Quadrants into Action  
• Quadrant 1 (Do) – Block immediate focus time in your calendar; eliminate all interruptions.  
• Quadrant 2 (Schedule) – Assign fixed time slots later in the week (e.g., “Tuesday 10–11 AM: Skill-building”).  
• Quadrant 3 (Delegate) – Identify teammates, tools, or services that can handle these tasks. Provide clear instructions and deadlines.  
• Quadrant 4 (Eliminate) – Remove from your to-do list; set timers or apps to curb these activities.

### Step 4: Weekly Rhythm & Review  
1. Every Sunday evening:  
   • Review and update your matrix.  
   • Confirm Quadrant 1 tasks for Monday.  
2. Daily quick-check (5 min):  
   • Are any new items urgent? Re-matrix them.  
3. End-of-day reflection (10 min):  
   • What moved from Quadrant 2 to Quadrant 1? Adjust schedule.

### Clarifying Questions  
1. What are your top 5 recurring tasks that feel both urgent and draining?  
2. Which activities you find yourself “wasting time” on?  
3. Do you have teammates or tools you can delegate to?  
4. How much uninterrupted focus time can you realistically commit per day?  

Answering these will help refine the matrix to your specific context and make your week dramatically more productive.

In [10]:
// Create SK Kernel with both general purpose and reasoning AI model services 
// Note: There are three AI Chat Completion services in this example
using System.Net.Http;

var aiHttpClient = new HttpClient();
aiHttpClient.Timeout = TimeSpan.FromSeconds(300);

var semanticKernelWithOpenSource = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion( // General Purpose Model
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion( // Reasoning Model
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "ReasoningAI") // Open-Source Local Model
    .AddOpenAIChatCompletion(
        modelId: "openai/gpt-oss-20b",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: null,
        serviceId: "OpenSourceAI",
        httpClient: aiHttpClient)
    .Build();

In [11]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ReasoningEffort = "High",
    MaxTokens = 8000,
    Temperature = 0.6, // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "OpenSourceAI" // Use the Open-Source Local Model
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernelWithOpenSource, // Use the Kernel with Open-Source Local Model service
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in 
    dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.

}
agentResponse.DisplayAs("text/markdown");

### Quick Clarification

Before we build a prioritization plan, I’d like to understand a few details about your typical week:

| Question | Why it matters |
|----------|----------------|
| What are the main activities you usually juggle (e.g., work projects, personal errands, learning goals)? | Helps identify the task pool. |
| Which tasks feel “unproductive” to you? Are they truly low‑value or just hard to start? | Clarifies whether the issue is task selection or execution. |
| Do you have any fixed deadlines, meetings, or obligations that cannot be moved? | Sets hard constraints. |
| How much time do you realistically have available each day for focused work? | Determines capacity limits. |
| Are there recurring habits or patterns that drain your energy (e.g., excessive email checking, social media)? | Identifies potential waste to eliminate. |
| What are your short‑term and long‑term goals (e.g., career advancement, health, skill development)? | Aligns priorities with broader objectives. |

Once you share these details, we can tailor the Eisenhower Decision Prioritization Matrix to your context. In the meantime, here’s a generic template you can start filling in:

---

### Eisenhower Decision Prioritization Matrix (Template)

| **Urgent / Important** | **Not Urgent / Important** |
|------------------------|---------------------------|
| *List tasks that need immediate attention and align with your goals.* | *List strategic, goal‑driven tasks you can schedule for later.* |
| **Not Urgent / Not Important** | **Urgent / Not Important** |
| *Tasks that are low‑value or distractions.* | *Interruptions you can delegate or eliminate.* |

**How to use:**

1. **Collect all tasks** for the week in a single list.
2. **Categorize each task** into one of the four quadrants above.
3. **Schedule**:  
   * Urgent/Important → Do first (today or tomorrow).  
   * Not Urgent/Important → Block dedicated time in your calendar.  
   * Urgent/Not Important → Delegate if possible; otherwise, set a short “quick‑action” slot.  
   * Not Urgent/Not Important → Consider dropping or postponing indefinitely.
4. **Review daily**: Move tasks that become urgent into the top row as needed.

---

### Suggested Weekly Structure (Based on Eisenhower’s Approach)

| Time Block | Focus Area | Key Activities |
|------------|------------|----------------|
| **Morning (2–3 hrs)** | High‑Impact Work | Deep‑work on *Not Urgent / Important* tasks (e.g., project development, learning). |
| **Mid‑Morning (30 min)** | Quick Actions | Handle *Urgent / Not Important* items (e.g., quick emails, scheduling). |
| **Lunch Break** | Recharge | Physical activity or mindfulness to reset energy. |
| **Afternoon (2 hrs)** | Administrative / Urgent | Finish *Urgent / Important* tasks that slipped into the day. |
| **Late Afternoon (30 min)** | Planning / Reflection | Review progress, update the matrix, prepare for next day. |
| **Evening** | Personal / Low‑Priority | Leisure, hobbies, or low‑effort tasks (e.g., reading). |

---

### Quick Wins to Reduce Unproductivity

1. **Batch similar tasks** (e.g., all emails at once) to reduce context switching.
2. **Set a “no‑meeting” hour** each day for deep work.
3. **Use the 2‑minute rule**: if a task < 2 min, do it immediately; otherwise, schedule.
4. **Limit digital distractions**: turn off non‑essential notifications during focused blocks.

---

### Next Steps

1. **Fill in the matrix** with your specific tasks and share it back if you’d like a more detailed schedule.
2. **Identify one “unproductive habit”** (e.g., endless email checking) and commit to a concrete change for the week.
3. **Schedule a brief check‑in** (e.g., Friday afternoon) to assess what worked and adjust the plan for next week.

Feel free to provide the missing details, and I’ll help refine this framework into a personalized action plan!

In [ ]:
// Helper method to invoke the chat completion agent with a list of prompts
private static async Task InvokeChatCompletionAgentWithTemplateAsync(ChatCompletionAgent agent, string[] prompts)
{
    ChatHistory chat = [];

    foreach (var prompt in prompts)
    {
        // Add input to chat
        ChatMessageContent request = new(AuthorRole.User, prompt);
        chat.Add(request);
        // Show as Markdown
        request.Content.DisplayAs("text/markdown");

        KernelArguments? arguements = null;

        // Process agent response
        await foreach (ChatMessageContent response in agent.InvokeAsync(chat, arguements))
        {
            chat.Add(response);
            response.Content.DisplayAs("text/markdown");
        }
    }
}

// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem how Dwight Eisenhower approached strategic problems and 
when appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = "An agent that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.";

string[] prompts = 
[
"How should I prioritize my typical unproductive day?",
"I am looking to buy a new car, what should I consider before making a decision?",
];

// Instruction based template always processed by KernelPromptTemplateFactory
ChatCompletionAgent agent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

await InvokeChatCompletionAgentWithTemplateAsync(agent, prompts);